### Time-dependent Problem

Here we extend the Laplace problem from the first example by a time variable. This leads to the heat equation:

\begin{align*}
    \partial_t u -0.1\Delta u &= 1.0 &&\text{ in } (0, 2) \times \Omega \\
    u &= 0 , &&\text{ on } (0, 2) \times \partial \Omega \\
    u(0, \cdot) &= 0 &&\text{ in } \Omega
\end{align*}

with $\Omega = (0, 1) \times (0, 1)$.

In [ ]:
# This block is for GPU selection. Please execute.
import pathlib
import os
user = int(str(pathlib.Path().resolve())[22:24])
os.environ["CUDA_VISIBLE_DEVICES"]= str(user % 4)

In [ ]:

import torch
import math

Next we implement the *Spaces* that appear in the problem:

In [2]:
import torchphysics as tp
X = tp.spaces.R2('x')
U = tp.spaces.R1('u')
T = tp.spaces. # TODO: Add the time variable "t" of dimension 1 

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Now we define our domain. The domain $\Omega$ is already completed, here you have to create the time interval and the Cartesian product of both.

In [3]:
omega = tp.domains.Parallelogram(X, [0,0], [1,0], [0,1])
time_interval = tp.domains.Interval(T, ) # TODO: Add the bounds of the Interval (0, 2)
product_domain = time_interval # TODO: Create the product domain of time and space. Products are define with: *

Next we need to create some points, this is done by the *Sampler*. Here we need 3, one inside the domain, one for the boundary and one for the initial condition.

In [4]:
# TODO: Add the product domain of time and space:
inner_sampler = tp.samplers.RandomUniformSampler(, n_points=25000) 

# The boundary sampler is done already.
bound_sampler = tp.samplers.RandomUniformSampler(time_interval*omega.boundary, n_points=10000)

# Currently only the left interval side {0} is passed in the initial sampler. 
# TODO: Create the product domain with omega:
initial_sampler = tp.samplers.RandomUniformSampler(time_interval.boundary_left, n_points=5000)

The neural network that learns the solution gets the time and space variable as an input and outputs the solution u. Add the correct spaces.

**Hint**: A product space is also defined with *

In [5]:
# TODO: Add the spaces
model = tp.models.FCN(input_space=, output_space=, hidden=(30,30,30))

Now, we have to transform our mathematical conditions given by our PDE into corresponding training conditions.

First we handle the PDE-condition itself. Here, you have to finish the residual function.

In [6]:
def pde_residual(u, x, t):
    # TODO: Pass in the correct variables for the derivative computation 
    # as the second argument below:
    return tp.utils.grad(u, ) - 0.1*tp.utils.laplacian(u, ) - 1.0

pde_cond = tp.conditions.PINNCondition(model, inner_sampler, pde_residual)

The boundary condition is already done, since it is the same as in the first example.

In [11]:
def boundary_residual(u):
    return u 

boundary_cond = tp.conditions.PINNCondition(model, bound_sampler, 
                                            boundary_residual, weight=100.0)

We now also need the initial condition. 

In [ ]:
# TODO: Implement the residual for the initial condition:
def initial_residual(u):
    return 

initial_cond = tp.conditions.PINNCondition() # TODO: Add the model, correct sampler and residual function

Before the training we collect all conditions and choose our training procedure:

In [12]:
optim = tp.OptimizerSetting(torch.optim.Adam, lr=0.005)
# TODO: Collect all conditions as a list and pass them to the solver as the
# first argument. 
solver = tp.solver.Solver([.,.], optimizer_setting=optim) 

Start the training:

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import RichProgressBar
from pytorch_lightning.callbacks.progress.rich_progress import RichProgressBarTheme
trainer = pl.Trainer(devices=1, accelerator="gpu", # use one GPU
                     max_steps=5000, # iteration number
                     benchmark=True, # faster if input batch has constant size
                     logger=False, # for writting into tensorboard
                     callbacks=[RichProgressBar(refresh_rate=20, theme=RichProgressBarTheme(metrics= "black", metrics_format='.7f'))], # show training process
                     enable_checkpointing=False) # saving checkpoints
trainer.fit(solver)

We can plot the solution, for two different time points:

In [ ]:
plot_sampler = tp.samplers.PlotSampler(plot_domain=omega, n_points=2000, data_for_other_variables={"t": 0.1})
fig = tp.utils.plot(model, lambda u : u, plot_sampler)


plot_sampler = tp.samplers.PlotSampler(plot_domain=omega, n_points=2000, data_for_other_variables={"t": 2.0})
fig = tp.utils.plot(model, lambda u : u, plot_sampler)

In [ ]:
# We can also animate the solution over time
anim_sampler = tp.samplers.AnimationSampler(omega, time_interval, 200, n_points=1000)
fig, anim = tp.utils.animate(model, lambda u: u, anim_sampler, ani_speed=10, angle=[30, 220])
anim.save('heat-eq.gif')
# You should be able to see a tab with a folder on the left side of your screen. There you can find the gif and can watch it.